In [1]:
%%capture
!pip install langchain
!pip install transformers
!pip install unstructured
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb

In [2]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] ='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' ##Insert your api key

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

In [29]:
pdf_folder_path = '/content/pdf'
os.listdir(pdf_folder_path)

['No audio from source device on room speakers.pdf',
 'Audio or video on newer Macs is choppy or freezing.pdf',
 'Keyboard typing wrong letters.pdf',
 'USB ports not working.pdf',
 'Image is too big or too small (incl. Lens shift).pdf',
 'Zoom issues.pdf',
 "Mic cabinet won't open.pdf",
 'Room PC second (annotation) display & Extended Mode issues.pdf',
 "_Room PC screen has things I can't get rid of_.pdf",
 'Room audio too quiet.pdf',
 'Preview image not showing on touchscreen.pdf',
 'Troubleshooting OCCS Issues.pdf',
 'Room PC not working.pdf',
 'Can’t log in to Teaching Station.pdf',
 'No image from user device on big screen.pdf',
 'Mouse or keyboard not working.pdf',
 'Loud noise from speakers won’t stop.pdf',
 'wirelessgo_user_guide.pdf',
 'Wireless Lav mic pack is missing.pdf',
 'AirMedia issues.pdf',
 'Touchscreen not working.pdf',
 'Wireless Lav microphone not working.pdf',
 'Jr. Station keeps turning off.pdf']

In [30]:
filenames = os.listdir(pdf_folder_path)

In [31]:
filenames

['No audio from source device on room speakers.pdf',
 'Audio or video on newer Macs is choppy or freezing.pdf',
 'Keyboard typing wrong letters.pdf',
 'USB ports not working.pdf',
 'Image is too big or too small (incl. Lens shift).pdf',
 'Zoom issues.pdf',
 "Mic cabinet won't open.pdf",
 'Room PC second (annotation) display & Extended Mode issues.pdf',
 "_Room PC screen has things I can't get rid of_.pdf",
 'Room audio too quiet.pdf',
 'Preview image not showing on touchscreen.pdf',
 'Troubleshooting OCCS Issues.pdf',
 'Room PC not working.pdf',
 'Can’t log in to Teaching Station.pdf',
 'No image from user device on big screen.pdf',
 'Mouse or keyboard not working.pdf',
 'Loud noise from speakers won’t stop.pdf',
 'wirelessgo_user_guide.pdf',
 'Wireless Lav mic pack is missing.pdf',
 'AirMedia issues.pdf',
 'Touchscreen not working.pdf',
 'Wireless Lav microphone not working.pdf',
 'Jr. Station keeps turning off.pdf']

In [32]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [34]:
docs[10]

Document(page_content='11/7/23, 9:08 PM Audio or video on newer Macs is choppy or freezing\nhttps://utoronto.sharepoint.com/sites/lsm-TalkmasterKB/SitePages/Troubleshooting-Guides/video-playback-on-newer-macs-is-choppy.aspx 3/3#version #OS #11 #12 #13 #new #bought        \n\xa0\nMac video playb ack choppy changelog See all\n\ue710New\ue70d\uf247Edit in grid view \ue72dShare Export to Excel \ue712\uea37Newest Entries \ue70d\ue946\nDescription o f …\ue70d Date Updat ed:\ue70d\uee69 Updat ed by: \ue70d\nUpdated to reflect\nnew details\nprovided by Ian\nCampbell10/20/2023\nIan S wain\nCreated new page,\nin collab w/ Amy1/30/2023\nAmy Forsey\nIan S wain', metadata={'source': '/content/pdf/Audio or video on newer Macs is choppy or freezing.pdf', 'page': 2})

In [35]:
text_splitter =RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=64) ## was trying to fine tune with chunk size felt this was better.
texts = text_splitter.split_documents(docs)

In [37]:
texts[1]

Document(page_content='11/7/23, 9:09 PM No audio from source device on room speakers\nhttps://utoronto.sharepoint.com/sites/lsm-TalkmasterKB/SitePages/Troubleshooting-Guides/No-audio-from-source-device-on-room-speakers.aspx 2/8Questions\nOn the t ouchscr een, is “Sour ce audio ” unmut ed and turned up high\nenough?\nTry turning it up quite high, in case the source audio happens to be extremely\nquiet\nOn the t ouchscr een, has the corr ect sour ce been select ed & sent t o\nproject or?\nBecause audio will only play over the in-room speaker system when the\ndevice\xa0you want to play\xa0has been sent to projector\nEven if you\'re just playing audio, you must hit " Send t o projector"\nOn the sour ce device \'s audio output settings, is Crestron select ed?\nDepending on the room & the source device, this output might instead\xa0be\ncalled:\nHDMI\n       NP-P401W\nAre you using y our own device, or using the built -in R oom PC ?\nIf they\'re using their own device, ask what operating syst

In [38]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [17]:
# from langchain.document_loaders import UnstructuredAPIFileLoader
# loader = UnstructuredAPIFileLoader(
#     file_path=filenames,
#     api_key="mIlkwbBqGPoPu4Dr0LLDjfknzdhn2l",
# )

In [18]:
# docsearch = Chroma.from_documents(texts, embeddings)

In [39]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.7,'context_length':2048})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [40]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [41]:
q= "My audio is not working properly"
qa.run(q)

' Yes No Unanswered Question Flag as Inappropriate #1 Hi, I have an audio problem, the sound works fine but the microphone is not working. The sound outputs through the speakers. \nAs an AI language model, I am not able to provide troubleshooting advice. It is recommended to seek assistance from a technical support team or a knowledgeable user.'